In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url = 'https://www.boxofficemojo.com/intl/uk/yearly/'

In [ ]:
# Access the webpage content
r = requests.get(url)

In [ ]:
# Parse the HTML page
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
# Choose the relevant table
table = soup.find_all('table')[4]

In [ ]:
# Parse and store the data of every table row
lst = []
for row in table.find_all('tr'):
    s = pd.Series([data.text for data in row.find_all('td')])
    lst.append(s)

In [ ]:
# Concatenate the data in a Pandas DataFrame and place the first row of the DataFrame as header. Use the .head() method to check your DataFrame.
data = pd.concat(lst, axis=1).T

# Grab the first row for the header
new_header = data.iloc[0]

# Take the data less the header row
data = data[1:]

# Set the header row as the df header
data.columns = new_header

data.head()

In [ ]:
# Q1
data.Distributor.value_counts().plot(kind='bar', title='Films per distributor')
plt.show()

In [ ]:
# Q2

# Remove the £ symbol and transform strings to integers
data['Gross'] = data['Gross'].apply(lambda x: int(x[1:].replace(',', '')))

# Group the data by Distributor and add the Gross value of their movies
data.groupby('Distributor')['Gross'].sum().sort_values(ascending=False).plot(kind='bar', title='Gross earning by distributor')

plt.show()